In [28]:
# Cell 1: Dissolve Street Centerlines (unchanged)
import arcpy
 
# Access current ArcGIS Pro project and map
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap
 
# Input layer name (update if different in the map)
input_layer_name = "CAMSCenterlines"
output_layer_name = "CAMSCenterlines_Dissolved"
 
# Check if the layer exists in the map
input_layer = None
for lyr in map_.listLayers():
    if lyr.name == input_layer_name:
        input_layer = lyr
        break
 
if not input_layer:
    raise ValueError(f"Layer '{input_layer_name}' not found in the current map.")
 
# Get layer path from the source
input_path = input_layer.dataSource
 
# Get the geodatabase or folder path
if input_path.endswith(".shp"):
    output_path = input_path.replace(".shp", "_Dissolved.shp")
else:
    workspace = arcpy.env.workspace = arcpy.Describe(input_path).path
    output_path = arcpy.Describe(workspace).catalogPath + f"\\{output_layer_name}"
 
# Perform dissolve by FullName
arcpy.management.Dissolve(
    in_features=input_path,
    out_feature_class=output_path,
    dissolve_field="FullName",
    statistics_fields="",
    multi_part="SINGLE_PART",
    unsplit_lines="DISSOLVE_LINES"
)
 
print(f"✅ Dissolved shapefile created at: {output_path}")
 

✅ Dissolved shapefile created at: C:\Users\JLin\Downloads\CAMSCenterlines (1)\CAMSCenterlines_Dissolved.shp


In [29]:
# Cell 2: Get All Rows from Standalone Table
import arcpy
import difflib
 
# Configuration
table_name = "Section List"
dissolved_shapefile = "CAMSCenterlines_Dissolved"
 
# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap
 
# Find table
table_path = None
for table in map_.listTables():
    if table.name == table_name:
        table_path = arcpy.Describe(table).catalogPath
        break
 
if not table_path:
    raise ValueError(f"Table '{table_name}' not found")
 
# Find dissolved shapefile
dissolved_path = None
for lyr in map_.listLayers():
    if lyr.name == dissolved_shapefile:
        dissolved_path = arcpy.Describe(lyr).catalogPath
        break
 
if not dissolved_path:
    raise ValueError(f"Dissolved shapefile '{dissolved_shapefile}' not found")
 
# Get field names from table
field_names = [f.name for f in arcpy.ListFields(table_path)]
print(f"Available table fields: {field_names}")
 
# Find the name field
name_field_table = None
for field in field_names:
    if 'name' in field.lower():
        name_field_table = field
        break
 
if not name_field_table:
    raise ValueError("Could not find 'Name' field in table")
 
# Get ALL rows from table
table_rows = []
row_count = int(arcpy.management.GetCount(table_path)[0])
print(f"Total rows in table: {row_count}")
 
with arcpy.da.SearchCursor(table_path, [name_field_table]) as cursor:
    for i, row in enumerate(cursor):
        street_name = str(row[0]).strip()
        table_rows.append((i+1, street_name))
 
print(f"Retrieved {len(table_rows)} rows from table:")
# Show first 10 and last 5 for preview
for i, (row_num, street) in enumerate(table_rows):
    if i < 10:
        print(f"  Row {row_num}: '{street}'")
    elif i == 10:
        print(f"  ... (showing first 10 of {len(table_rows)} rows)")
        break
 
if len(table_rows) > 10:
    print(f"  Last few rows:")
    for row_num, street in table_rows[-5:]:
        print(f"  Row {row_num}: '{street}'")
 
# Fuzzy matching function
def fuzzy_match(target, choices, threshold=0.7):
    target_norm = str(target).lower().replace('.', '').strip()
    best_match = None
    best_score = 0
   
    for choice in choices:
        if not choice:
            continue
        choice_norm = str(choice).lower().replace('.', '').strip()
        score = difflib.SequenceMatcher(None, target_norm, choice_norm).ratio()
       
        if score > best_score and score >= threshold:
            best_match = choice
            best_score = score
   
    return best_match, best_score
 
# Get all street names from dissolved shapefile
dissolved_streets = set()
with arcpy.da.SearchCursor(dissolved_path, ["FullName"]) as cursor:
    for row in cursor:
        if row[0]:
            dissolved_streets.add(str(row[0]).strip())
 
# Match each table row to dissolved streets
matched_streets = []
print(f"\n🔍 Matching table streets to dissolved shapefile:")
for row_num, target_street in table_rows:
    match, score = fuzzy_match(target_street, dissolved_streets)
    if match:
        if row_num <= 20:  # Only show detailed matching for first 20 rows
            print(f"  Row {row_num}: '{target_street}' → '{match}' (score: {score:.2f})")
        matched_streets.append((row_num, target_street, match))
    else:
        if row_num <= 20:  # Only show detailed matching for first 20 rows
            print(f"  Row {row_num}: '{target_street}' → No match found, using exact name")
        matched_streets.append((row_num, target_street, target_street))
 
print(f"\n✅ Matched {len(matched_streets)} streets")

# =============================================================================
# Cell 2b: Automatic Threshold‑Stepping Fuzzy Match Retry
# -----------------------------------------------------------------------------
# • Requires variables created in Cell 2:
#       - table_rows        (list of (row_num, street_name))
#       - dissolved_streets (set of street names from the dissolved layer)
#       - matched_streets   (list of tuples from first pass)
#       - fuzzy_match()     (function defined in Cell 2)
#
# • Loops over a sequence of descending thresholds (e.g. 0.65, 0.60, … 0.45)
#   and re‑tries any still‑unmatched rows at each step.
#
# • Stops early if all rows are matched.
# =============================================================================

# --- CONFIGURATION -----------------------------------------------------------
thresholds = [0.65, 0.60, 0.55, 0.50, 0.45]   # add more if needed
preview_limit = 15   # rows to echo for inspection at each threshold

# --- Helper: find currently unmatched rows -----------------------------------
def current_unmatched():
    matched_set = {r for r, _, _ in matched_streets}
    return [(r, s) for r, s in table_rows if r not in matched_set]

# --- Main loop ---------------------------------------------------------------
for th in thresholds:
    still_unmatched = current_unmatched()
    if not still_unmatched:
        print("🎉 All rows matched – no need for lower thresholds.")
        break

    print("\n" + "="*72)
    print(f"🔄  Retrying {len(still_unmatched)} rows at threshold {th:.2f}")
    new_hits = 0

    for idx, (row_num, street) in enumerate(still_unmatched, 1):
        match, score = fuzzy_match(street, dissolved_streets, threshold=th)
        if match:
            matched_streets.append((row_num, street, match))
            new_hits += 1
            if idx <= preview_limit:
                print(f"  Row {row_num}: '{street}' → '{match}'  (score {score:.2f})")
        elif idx <= preview_limit:
            print(f"  Row {row_num}: '{street}' → still no match")

    print(f"✅  Added {new_hits} new matches at threshold {th:.2f}")

# --- Final summary -----------------------------------------------------------
total = len(table_rows)
print("\n" + "-"*72)
print(f"🏁 Matching complete – {len(matched_streets)}/{total} rows matched")
remaining = total - len(matched_streets)
if remaining:
    print(f"⚠️  {remaining} row(s) still unmatched "
          f"(consider manual review or extend threshold list)")
else:
    print("🎉 Every row now has a fuzzy‑match candidate")
print("-"*72)

Available table fields: ['OBJECTID', 'Sec_ID', 'Name', 'From_', 'To', 'Length', 'Width', 'Lanes', 'Layer_Name']
Total rows in table: 552
Retrieved 552 rows from table:
  Row 1: 'Abbey St.'
  Row 2: 'Abbey St.'
  Row 3: 'Abbey St.'
  Row 4: 'Aileron Ave.'
  Row 5: 'Aileron Ave.'
  Row 6: 'Aileron Ave.'
  Row 7: 'Aileron Ave.'
  Row 8: 'Aileron Ave.'
  Row 9: 'Aileron Ave.'
  Row 10: 'Albert St.'
  ... (showing first 10 of 552 rows)
  Last few rows:
  Row 548: 'Workman St.'
  Row 549: 'Workman St.'
  Row 550: 'Workman St.'
  Row 551: 'Workman St.'
  Row 552: 'Workman St.'

🔍 Matching table streets to dissolved shapefile:
  Row 1: 'Abbey St.' → 'Abbey St' (score: 1.00)
  Row 2: 'Abbey St.' → 'Abbey St' (score: 1.00)
  Row 3: 'Abbey St.' → 'Abbey St' (score: 1.00)
  Row 4: 'Aileron Ave.' → 'Aileron Ave' (score: 1.00)
  Row 5: 'Aileron Ave.' → 'Aileron Ave' (score: 1.00)
  Row 6: 'Aileron Ave.' → 'Aileron Ave' (score: 1.00)
  Row 7: 'Aileron Ave.' → 'Aileron Ave' (score: 1.00)
  Row 8: 'Ail

In [30]:
# Cell 3: Create Segments for All Matched Streets
import arcpy
import os
 
# Configuration
dissolved_shapefile = "CAMSCenterlines_Dissolved"
output_segments = "AllRows_Matched_Segments"
 
# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap
 
# Find dissolved shapefile
dissolved_path = None
for lyr in map_.listLayers():
    if lyr.name == dissolved_shapefile:
        dissolved_path = arcpy.Describe(lyr).catalogPath
        break
 
if not dissolved_path:
    raise ValueError(f"Dissolved shapefile '{dissolved_shapefile}' not found")
 
# Setup workspace
workspace = os.path.dirname(dissolved_path)
output_path = os.path.join(workspace, f"{output_segments}.shp")
 
print(f"Creating segments for {len(matched_streets)} streets:")
print(f"Output will be saved to: {output_path}")
 
# Estimate processing time
if len(matched_streets) > 50:
    print(f"⏱️  Processing {len(matched_streets)} streets - this may take a few minutes...")
elif len(matched_streets) > 100:
    print(f"⏱️  Processing {len(matched_streets)} streets - this may take 5-10 minutes...")
 
# Create output feature class with same schema as dissolved shapefile
spatial_ref = arcpy.Describe(dissolved_path).spatialReference
arcpy.management.CreateFeatureclass(
    out_path=workspace,
    out_name=output_segments,
    geometry_type="POLYLINE",
    template=dissolved_path,
    spatial_reference=spatial_ref
)
 
# Add additional fields for tracking (shortened for shapefile compatibility)
arcpy.management.AddField(output_path, "RowNum", "LONG")
arcpy.management.AddField(output_path, "OrigName", "TEXT", field_length=50)
arcpy.management.AddField(output_path, "MatchName", "TEXT", field_length=50)
 
# Get original fields from dissolved shapefile
original_fields = [f.name for f in arcpy.ListFields(dissolved_path)
                   if f.type not in ['OID', 'Geometry']]
 
# Process each matched street
total_features = 0
with arcpy.da.InsertCursor(output_path, ["SHAPE@", "RowNum", "OrigName", "MatchName"] + original_fields) as insert_cursor:
   
    for row_num, original_street, matched_street in matched_streets:
        if row_num <= 20:  # Only show detailed processing for first 20 rows
            print(f"\n  Processing Row {row_num}: '{matched_street}'")
        elif row_num % 50 == 0:  # Show progress every 50 rows for large datasets
            progress = (row_num / len(matched_streets)) * 100
            print(f"\n  Progress: {progress:.1f}% complete (Row {row_num}/{len(matched_streets)})")
       
        # Select features from dissolved shapefile
        field_delimited = arcpy.AddFieldDelimiters(dissolved_path, "FullName")
        safe_street_name = matched_street.replace("'", "''")
        where_clause = f"{field_delimited} = '{safe_street_name}'"
       
        # Count matching features
        feature_count = 0
        with arcpy.da.SearchCursor(dissolved_path, ["SHAPE@"] + original_fields, where_clause) as cursor:
            for feature_row in cursor:
                # Insert the feature with additional tracking fields
                new_row = [feature_row[0], row_num, original_street, matched_street] + list(feature_row[1:])
                insert_cursor.insertRow(new_row)
                feature_count += 1
                total_features += 1
       
        if feature_count > 0:
            if row_num <= 20:  # Only show detailed messages for first 20 rows
                print(f"    Found {feature_count} feature(s)")
        elif row_num <= 20:  # Only show detailed messages for first 20 rows to avoid spam
            print(f"    ⚠️  No features found for '{matched_street}'")
 
# Add to map
map_.addDataFromPath(output_path)
 
print(f"\n✅ Created all-rows segments: {output_segments}")
print(f"Total features extracted: {total_features}")
 
# Show processing summary
print(f"📊 Processing summary:")
print(f"  - Table rows processed: {len(table_rows)}")
print(f"  - Streets successfully matched: {len(matched_streets)}")
print(f"  - Total features extracted: {total_features}")

Creating segments for 552 streets:
Output will be saved to: C:\Users\JLin\Downloads\CAMSCenterlines (1)\AllRows_Matched_Segments.shp
⏱️  Processing 552 streets - this may take a few minutes...

  Processing Row 1: 'Abbey St'
    Found 2 feature(s)

  Processing Row 2: 'Abbey St'
    Found 2 feature(s)

  Processing Row 3: 'Abbey St'
    Found 2 feature(s)

  Processing Row 4: 'Aileron Ave'
    Found 2 feature(s)

  Processing Row 5: 'Aileron Ave'
    Found 2 feature(s)

  Processing Row 6: 'Aileron Ave'
    Found 2 feature(s)

  Processing Row 7: 'Aileron Ave'
    Found 2 feature(s)

  Processing Row 8: 'Aileron Ave'
    Found 2 feature(s)

  Processing Row 9: 'Aileron Ave'
    Found 2 feature(s)

  Processing Row 10: 'Albert St'
    Found 2 feature(s)

  Processing Row 11: 'Albert St'
    Found 2 feature(s)

  Processing Row 12: 'Aldgate Ave'
    Found 3 feature(s)

  Processing Row 13: 'Aldgate Ave'
    Found 3 feature(s)

  Processing Row 14: 'Aldgate Ave'
    Found 3 feature(s)

  

In [31]:
# Cell 4: Create Intersections for All Segments
import arcpy
import os
 
# Configuration
matched_segments_layer = "AllRows_Matched_Segments"
dissolved_shapefile = "CAMSCenterlines_Dissolved"
output_intersections = "AllRows_Intersections"
tolerance = 0.1
 
# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap
 
# Find layers
matched_segments_path = None
dissolved_path = None
 
for lyr in map_.listLayers():
    if lyr.name == matched_segments_layer:
        matched_segments_path = arcpy.Describe(lyr).catalogPath
    elif lyr.name == dissolved_shapefile:
        dissolved_path = arcpy.Describe(lyr).catalogPath
 
if not matched_segments_path:
    raise ValueError(f"Layer '{matched_segments_layer}' not found")
if not dissolved_path:
    raise ValueError(f"Layer '{dissolved_shapefile}' not found")
 
# Setup workspace
workspace = os.path.dirname(matched_segments_path)
output_path = os.path.join(workspace, f"{output_intersections}.shp")
 
print(f"Finding intersections for all matched segments:")
print(f"Output: {output_path}")
 
if len(matched_streets) > 50:
    print(f"⏱️  Processing intersections for {len(matched_streets)} streets - this may take several minutes...")
 
# Run intersection analysis
arcpy.analysis.Intersect(
    in_features=[matched_segments_path, dissolved_path],
    out_feature_class=output_path,
    join_attributes="ALL",
    cluster_tolerance="",
    output_type="POINT"
)
 
# Add fields for intersection information
arcpy.management.AddField(output_path, "Street1", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "Street2", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "IntName", "TEXT", field_length=200)
 
# Get field names to find FullName fields
field_names = [f.name for f in arcpy.ListFields(output_path)]
fullname_fields = [f for f in field_names if "FullName" in f]
 
print(f"Found FullName fields: {fullname_fields}")
 
# Populate intersection fields and remove self-intersections
unique_intersections = []
intersection_count = 0
 
if len(fullname_fields) >= 2:
    with arcpy.da.UpdateCursor(output_path, ["SHAPE@", "RowNum"] + fullname_fields + ["Street1", "Street2", "IntName"]) as cursor:
        for row in cursor:
            point_geom = row[0]
            row_number = row[1]
            street1 = str(row[2]) if row[2] else ""
            street2 = str(row[3]) if row[3] else ""
           
            # Skip self-intersections (same street)
            if street1.lower() != street2.lower() and street1 and street2:
                int_name = f"{street1} & {street2}"
               
                # Check for duplicates within tolerance for the same row
                is_duplicate = False
                for existing_point, existing_name, existing_row in unique_intersections:
                    if existing_row == row_number:  # Same table row
                        distance = point_geom.distanceTo(existing_point)
                        if distance <= tolerance:
                            # Check if it's the same intersection (same streets)
                            existing_streets = set(existing_name.split(' & '))
                            current_streets = set([street1.lower(), street2.lower()])
                            if current_streets == existing_streets:
                                is_duplicate = True
                                break
               
                if not is_duplicate:
                    unique_intersections.append((point_geom, int_name, row_number))
                    cursor.updateRow([point_geom, row_number] + list(row[2:-3]) + [street1, street2, int_name])
                    intersection_count += 1
                else:
                    cursor.deleteRow()
            else:
                cursor.deleteRow()
 
# Add to map
map_.addDataFromPath(output_path)
 
print(f"✅ Created intersection points: {output_intersections}")
print(f"Found {intersection_count} unique intersections across all {len(table_rows)} table rows")
 
# Show summary by row (first 10 and summary stats)
print(f"\n📊 Intersections by table row (showing first 10):")
row_counts = {}
with arcpy.da.SearchCursor(output_path, ["RowNum", "IntName"]) as cursor:
    for row in cursor:
        row_num = row[0]
        if row_num not in row_counts:
            row_counts[row_num] = 0
        row_counts[row_num] += 1
 
# Show first 10 rows and summary statistics
shown_count = 0
for row_num in sorted(row_counts.keys()):
    count = row_counts[row_num]
    original_name = next((orig for r, orig, match in matched_streets if r == row_num), "Unknown")
    if shown_count < 10:
        print(f"  Row {row_num} ({original_name}): {count} intersections")
        shown_count += 1
    elif shown_count == 10:
        print(f"  ... (showing first 10 of {len(row_counts)} rows with intersections)")
        break
 
if len(row_counts) > 0:
    intersection_counts = list(row_counts.values())
    avg_intersections = sum(intersection_counts) / len(intersection_counts)
    max_intersections = max(intersection_counts)
    min_intersections = min(intersection_counts)
    print(f"\n📈 Intersection statistics:")
    print(f"  - Rows with intersections: {len(row_counts)}/{len(table_rows)}")
    print(f"  - Average intersections per row: {avg_intersections:.1f}")
    print(f"  - Max intersections (single row): {max_intersections}")
    print(f"  - Min intersections (single row): {min_intersections}")

Finding intersections for all matched segments:
Output: C:\Users\JLin\Downloads\CAMSCenterlines (1)\AllRows_Intersections.shp
⏱️  Processing intersections for 552 streets - this may take several minutes...
Found FullName fields: ['FullName', 'FullName_1']
✅ Created intersection points: AllRows_Intersections
Found 6247 unique intersections across all 552 table rows

📊 Intersections by table row (showing first 10):
  Row 1 (Abbey St.): 5 intersections
  Row 2 (Abbey St.): 5 intersections
  Row 3 (Abbey St.): 5 intersections
  Row 4 (Aileron Ave.): 16 intersections
  Row 5 (Aileron Ave.): 16 intersections
  Row 6 (Aileron Ave.): 16 intersections
  Row 7 (Aileron Ave.): 16 intersections
  Row 8 (Aileron Ave.): 16 intersections
  Row 9 (Aileron Ave.): 16 intersections
  Row 10 (Albert St.): 9 intersections
  ... (showing first 10 of 523 rows with intersections)

📈 Intersection statistics:
  - Rows with intersections: 523/552
  - Average intersections per row: 11.9
  - Max intersections (sin

In [32]:
# Cell 5: Create Split Segments for All Streets
import arcpy
import os
 
# Configuration
matched_segments_layer = "AllRows_Matched_Segments"
intersections_layer = "AllRows_Intersections"
output_segments = "AllRows_Split_Segments"
tolerance = 1.0
 
# Get current project
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap
 
# Find layers
matched_segments_path = None
intersections_path = None
 
for lyr in map_.listLayers():
    if lyr.name == matched_segments_layer:
        matched_segments_path = arcpy.Describe(lyr).catalogPath
    elif lyr.name == intersections_layer:
        intersections_path = arcpy.Describe(lyr).catalogPath
 
if not matched_segments_path:
    raise ValueError(f"Layer '{matched_segments_layer}' not found")
if not intersections_path:
    raise ValueError(f"Layer '{intersections_layer}' not found")
 
# Setup workspace
workspace = os.path.dirname(matched_segments_path)
spatial_ref = arcpy.Describe(matched_segments_path).spatialReference
output_path = os.path.join(workspace, f"{output_segments}.shp")
 
print(f"Splitting segments for all matched streets:")
print(f"Output: {output_path}")
 
if len(matched_streets) > 50:
    print(f"⏱️  Splitting {len(matched_streets)} streets into segments - this may take 10-15 minutes...")
elif len(matched_streets) > 20:
    print(f"⏱️  Splitting {len(matched_streets)} streets into segments - this may take a few minutes...")
 
# Load intersection points by row number
intersections_by_row = {}
with arcpy.da.SearchCursor(intersections_path, ["SHAPE@", "RowNum", "Street1", "Street2", "IntName"]) as cursor:
    for row in cursor:
        point_geom, row_number, street1, street2, int_name = row
        if street1 and street2 and int_name:
            if row_number not in intersections_by_row:
                intersections_by_row[row_number] = []
           
            intersections_by_row[row_number].append({
                'geometry': point_geom,
                'point': point_geom.firstPoint,
                'int_name': int_name,
                'street1': street1,
                'street2': street2
            })
 
print(f"Loaded intersections for {len(intersections_by_row)} table rows")
 
# Create output feature class
arcpy.management.CreateFeatureclass(
    out_path=workspace,
    out_name=os.path.basename(output_path).replace('.shp', ''),
    geometry_type="POLYLINE",
    spatial_reference=spatial_ref
)
 
# Add fields (shortened for shapefile compatibility)
arcpy.management.AddField(output_path, "RowNum", "LONG")
arcpy.management.AddField(output_path, "StreetName", "TEXT", field_length=50)
arcpy.management.AddField(output_path, "OrigName", "TEXT", field_length=50)
arcpy.management.AddField(output_path, "FromInt", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "ToInt", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "SegLen", "DOUBLE")
arcpy.management.AddField(output_path, "SegmentID", "TEXT", field_length=50)
 
# Copy original fields (with length limits for shapefiles)
original_fields = [f.name for f in arcpy.ListFields(matched_segments_path)
                   if f.type not in ['OID', 'Geometry'] and f.name.upper() not in ['SHAPE_LENGTH', 'SHAPE_AREA']]
 
for field_name in original_fields:
    if field_name not in ['RowNum', 'OrigName']:  # Skip fields we already added
        field_obj = arcpy.ListFields(matched_segments_path, field_name)[0]
        try:
            # Truncate field name to 10 characters for shapefile compatibility
            short_field_name = field_name[:10] if len(field_name) > 10 else field_name
            # Make sure the shortened name doesn't conflict with existing fields
            existing_fields = [f.name for f in arcpy.ListFields(output_path)]
            counter = 1
            original_short_name = short_field_name
            while short_field_name in existing_fields:
                short_field_name = f"{original_short_name[:8]}{counter:02d}"
                counter += 1
           
            arcpy.management.AddField(output_path, short_field_name, field_obj.type, field_length=min(field_obj.length, 254))
        except Exception as e:
            print(f"    Warning: Could not add field {field_name}: {e}")
            continue
 
def split_line_at_intersections(line_geom, intersections, tolerance):
    """Split a line at intersection points"""
    if not intersections:
        return [line_geom], [("START", "END")]
   
    # Get measures for intersection points on this line
    points_with_measure = []
    for intersection in intersections:
        try:
            measure = line_geom.measureOnLine(intersection['point'])
            # Only include points that are actually on the line (not just nearby)
            if 0 <= measure <= line_geom.length:
                points_with_measure.append((measure, intersection))
        except:
            continue
   
    # Sort by position along the line
    points_with_measure.sort(key=lambda x: x[0])
   
    # Remove duplicate measures (points too close together)
    filtered_points = []
    for measure, intersection in points_with_measure:
        if not filtered_points or abs(measure - filtered_points[-1][0]) > tolerance:
            filtered_points.append((measure, intersection))
   
    points_with_measure = filtered_points
   
    if not points_with_measure:
        return [line_geom], [("START", "END")]
   
    # Create segments
    segments = []
    segment_info = []
   
    # Build measures list: start, intersection points, end
    measures = [0.0]
    intersection_names = [None]
   
    for measure, intersection in points_with_measure:
        measures.append(measure)
        intersection_names.append(intersection['int_name'])
   
    measures.append(line_geom.length)
    intersection_names.append(None)
   
    # Create segments between consecutive measures
    for i in range(len(measures) - 1):
        start_measure = measures[i]
        end_measure = measures[i + 1]
       
        # Skip very short segments
        if end_measure - start_measure < tolerance:
            continue
       
        try:
            segment = line_geom.segmentAlongLine(start_measure, end_measure)
            if segment and segment.length > tolerance:
                segments.append(segment)
               
                from_int = intersection_names[i] if intersection_names[i] else "START"
                to_int = intersection_names[i + 1] if intersection_names[i + 1] else "END"
               
                segment_info.append((from_int, to_int))
        except Exception as e:
            continue
   
    # If no segments were created, return the original line
    if not segments:
        return [line_geom], [("START", "END")]
   
    return segments, segment_info
 
# Split segments for each row
total_created_segments = 0
 
with arcpy.da.SearchCursor(matched_segments_path, ["SHAPE@", "RowNum", "OrigName", "MatchName"] + [f for f in original_fields if f not in ['RowNum', 'OrigName']]) as cursor:
    for row in cursor:
        line_geom = row[0]
        row_number = row[1]
        original_name = row[2]
        street_name = row[3]
        other_attributes = row[4:]
       
        if row_number <= 20:  # Only show detailed processing for first 20 rows
            print(f"\nSplitting Row {row_number}: {street_name}")
        elif row_number % 50 == 0:  # Show progress every 50 rows for large datasets
            progress = (row_number / len(matched_streets)) * 100
            print(f"\nProgress: {progress:.1f}% complete (Row {row_number}/{len(matched_streets)})")
       
        # Get intersections for this row
        row_intersections = intersections_by_row.get(row_number, [])
       
        # Split the line at intersection points
        segments, segment_info = split_line_at_intersections(line_geom, row_intersections, tolerance)
        if len(segments) > 0:
            if row_number <= 20:  # Only show detailed messages for first 20 rows
                print(f"  Created {len(segments)} segments")
        elif row_number <= 20:  # Only show detailed messages for first 20 rows
            print(f"  ⚠️  No segments created (no intersections or very short street)")
       
        # Insert segments
        with arcpy.da.InsertCursor(output_path,
                                   ["SHAPE@", "RowNum", "StreetName", "OrigName", "FromInt", "ToInt", "SegLen", "SegmentID"] + [f for f in original_fields if f not in ['RowNum', 'OrigName']]) as insert_cursor:
           
            for i, (segment_geom, (from_int, to_int)) in enumerate(zip(segments, segment_info)):
                segment_id = f"Row{row_number}_{street_name}_Seg_{i + 1}"
               
                insert_cursor.insertRow([
                    segment_geom,
                    row_number,
                    street_name,
                    original_name,
                    from_int,
                    to_int,
                    segment_geom.length,
                    segment_id
                ] + list(other_attributes))
               
                if row_number <= 20:  # Only show detailed segment info for first 20 rows
                    print(f"    Segment {i+1}: {from_int} → {to_int} ({segment_geom.length:.1f} units)")
       
        total_created_segments += len(segments)
 
# Add to map
map_.addDataFromPath(output_path)
 
print(f"\n✅ Created split segments: {output_segments}")
print(f"Total segments created: {total_created_segments}")
if len(matched_streets) > 0:
    print(f"Average segments per street: {total_created_segments/len(matched_streets):.1f}")
 

Splitting segments for all matched streets:
Output: C:\Users\JLin\Downloads\CAMSCenterlines (1)\AllRows_Split_Segments.shp
⏱️  Splitting 552 streets into segments - this may take 10-15 minutes...
Loaded intersections for 523 table rows

Splitting Row 1: Abbey St
  Created 1 segments
    Segment 1: Abbey St & Common St → Abbey St & Central Ave (817.8 units)

Splitting Row 1: Abbey St
  Created 3 segments
    Segment 1: Abbey St & S Stimson Ave → Abbey St & Albert St (551.0 units)
    Segment 2: Abbey St & Albert St → Abbey St & Common St (669.1 units)
    Segment 3: Abbey St & Common St → Abbey St & Common St (2.6 units)

Splitting Row 2: Abbey St
  Created 1 segments
    Segment 1: Abbey St & Common St → Abbey St & Central Ave (817.8 units)

Splitting Row 2: Abbey St
  Created 3 segments
    Segment 1: Abbey St & S Stimson Ave → Abbey St & Albert St (551.0 units)
    Segment 2: Abbey St & Albert St → Abbey St & Common St (669.1 units)
    Segment 3: Abbey St & Common St → Abbey St & Co

Splitting Row 10: Albert St
  Created 3 segments
    Segment 1: Albert St & → Albert St & Central Ave (2.2 units)
    Segment 2: Albert St & Central Ave → Albert St & Abbey St (357.2 units)
    Segment 3: Albert St & Abbey St → Albert St & Old Valley Blvd (203.9 units)

Splitting Row 10: Albert St
  Created 3 segments
    Segment 1: Albert St & Main St → Albert St & (361.7 units)
    Segment 2: Albert St & → Albert St & Greenbriar Ln (183.1 units)
    Segment 3: Albert St & Greenbriar Ln → Albert St & Abbey St (138.3 units)

Splitting Row 11: Albert St
  Created 3 segments
    Segment 1: Albert St & → Albert St & Central Ave (2.2 units)
    Segment 2: Albert St & Central Ave → Albert St & Abbey St (357.2 units)
    Segment 3: Albert St & Abbey St → Albert St & Old Valley Blvd (203.9 units)

Splitting Row 11: Albert St
  Created 3 segments
    Segment 1: Albert St & Main St → Albert St & (361.7 units)
    Segment 2: Albert St & → Albert St & Greenbriar Ln (183.1 units)
    Segment 3: Al

    Segment 7: N Hacienda Blvd & Santo Oro Ave → N Hacienda Blvd & Ector St (266.7 units)
    Segment 8: N Hacienda Blvd & Ector St → N Hacienda Blvd & Sierra Vista Ct (244.3 units)
    Segment 9: N Hacienda Blvd & Sierra Vista Ct → N Hacienda Blvd & Giordano St (14.2 units)
    Segment 10: N Hacienda Blvd & Giordano St → N Hacienda Blvd & Elliot Ave (347.0 units)

Splitting Row 18: N Hacienda Blvd
  Created 9 segments
    Segment 1: N Hacienda Blvd & E Nelson Ave → N Hacienda Blvd & Nelson Ave (18.7 units)
    Segment 2: N Hacienda Blvd & Nelson Ave → N Hacienda Blvd & Prichard St (1045.5 units)
    Segment 3: N Hacienda Blvd & Prichard St → N Hacienda Blvd & E Temple Ave (781.5 units)
    Segment 4: N Hacienda Blvd & E Temple Ave → N Hacienda Blvd & (168.9 units)
    Segment 5: N Hacienda Blvd & → N Hacienda Blvd & Santo Oro Ave (320.7 units)
    Segment 6: N Hacienda Blvd & Santo Oro Ave → N Hacienda Blvd & Ector St (266.5 units)
    Segment 7: N Hacienda Blvd & Ector St → N Haciend

    Segment 4: N Hacienda Blvd & E Temple Ave → N Hacienda Blvd & (168.9 units)
    Segment 5: N Hacienda Blvd & → N Hacienda Blvd & Santo Oro Ave (320.7 units)
    Segment 6: N Hacienda Blvd & Santo Oro Ave → N Hacienda Blvd & Ector St (266.5 units)
    Segment 7: N Hacienda Blvd & Ector St → N Hacienda Blvd & Sierra Vista Ct (244.5 units)
    Segment 8: N Hacienda Blvd & Sierra Vista Ct → N Hacienda Blvd & Giordano St (13.2 units)
    Segment 9: N Hacienda Blvd & Giordano St → N Hacienda Blvd & Elliot Ave (352.3 units)

Splitting Row 20: N Hacienda Blvd
  Created 3 segments
    Segment 1: N Hacienda Blvd & → N Hacienda Blvd & Sierra Vista Ct (243.1 units)
    Segment 2: N Hacienda Blvd & Sierra Vista Ct → N Hacienda Blvd & Giordano St (15.3 units)
    Segment 3: N Hacienda Blvd & Giordano St → N Hacienda Blvd & Elliot Ave (104.0 units)

Splitting Row 20: N Hacienda Blvd
  Created 7 segments
    Segment 1: N Hacienda Blvd & → N Hacienda Blvd & Glendora Ave (196.0 units)
    Segment 2:

In [33]:
# =============================================================================
# Cell 6: Enhanced Match Table Requirements for All Rows 
# REPLACE YOUR EXISTING CELL 6 WITH THIS ENHANCED VERSION
# =============================================================================
import arcpy, os
import difflib

# -----------------------------------------------------------------------------#
# Configuration
# -----------------------------------------------------------------------------#
split_segments_layer   = "AllRows_Split_Segments"
table_name             = "Section List"
output_final_segments  = "Final_Matched_Segments"
unmatched_table_name   = "Unmatched_Rows"
# -----------------------------------------------------------------------------#

project = arcpy.mp.ArcGISProject("CURRENT")
m       = project.activeMap

#– locate layers/paths --------------------------------------------------------#
split_path = next(arcpy.Describe(l).catalogPath for l in m.listLayers()
                  if l.name == split_segments_layer)
table_path = next(arcpy.Describe(t).catalogPath   for t in m.listTables()
                  if t.name == table_name)

#– identify table fields ------------------------------------------------------#
field_names   = [f.name for f in arcpy.ListFields(table_path)]
name_field    = next(f for f in field_names if 'name' in f.lower())
from_field    = next(f for f in field_names if 'from' in f.lower())
to_field      = next(f for f in field_names if 'to'   in f.lower())

# Find Sec_ID field (accepts 'Sec_ID', 'SECID', etc.)
sec_id_field  = next((f for f in field_names if 'sec' in f.lower() and 'id' in f.lower()), None)
if not sec_id_field:
    raise ValueError("Cannot find a Sec_ID field in the table")

# Determine Sec_ID data-type - FIXED mapping
sec_obj = next(f for f in arcpy.ListFields(table_path) if f.name == sec_id_field)
print(f"Found Sec_ID field '{sec_id_field}' with type: {sec_obj.type}")

# Map field types to valid ArcGIS field types
type_mapping = {
    'Integer': 'LONG',
    'SmallInteger': 'LONG', 
    'OID': 'LONG',
    'String': 'TEXT',
    'Text': 'TEXT',
    'Double': 'DOUBLE',
    'Single': 'FLOAT',
    'Date': 'DATE'
}

sec_type_out = type_mapping.get(sec_obj.type, 'TEXT')  # Default to TEXT if unknown
print(f"Mapped to output type: {sec_type_out}")

#– read all table rows --------------------------------------------------------#
table_reqs = []
with arcpy.da.SearchCursor(table_path,
        [name_field, from_field, to_field, sec_id_field]) as cur:
    for i, (street, frm, to_, secid) in enumerate(cur, start=1):
        table_reqs.append({
            'row'       : i,
            'sec_id'    : secid,
            'street'    : str(street).strip(),
            'from_st'   : str(frm).strip(),
            'to_st'     : str(to_).strip()
        })

print(f"Processing {len(table_reqs)} table requirements:")

# =============================================================================
# ENHANCED MATCHING FUNCTIONS - NEW ADDITION
# =============================================================================

def smart_normalize(name):
    """Smart normalization that preserves key matching information"""
    if not name:
        return ""
    
    name = str(name).lower().strip()
    name = name.replace('.', '').replace(',', '')
    
    # Standardize abbreviations
    replacements = {
        ' avenue': ' ave', ' street': ' st', ' drive': ' dr',
        ' road': ' rd', ' boulevard': ' blvd', ' lane': ' ln',
        ' court': ' ct', ' place': ' pl', ' circle': ' cir'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    return ' '.join(name.split())  # Remove extra spaces

def extract_street_name_from_intersection(intersection_name):
    """Extract the cross street name from an intersection"""
    if not intersection_name or intersection_name == 'END':
        return intersection_name
    
    # Handle format like "Albert St & Central Ave" -> "Central Ave"
    if ' & ' in intersection_name:
        parts = intersection_name.split(' & ')
        if len(parts) == 2:
            return parts[1]  # Return the cross street
    
    return intersection_name

def smart_intersection_match(target_street, intersection_name, threshold=0.6):
    """Smart matching that looks at cross street names in intersections"""
    if not target_street or not intersection_name:
        return 0.0
    
    target_norm = smart_normalize(target_street)
    
    # Handle END specially
    if target_street.upper() == 'END' and intersection_name.upper() == 'END':
        return 1.0
    
    # Extract cross street from intersection name
    cross_street = extract_street_name_from_intersection(intersection_name)
    cross_norm = smart_normalize(cross_street)
    
    # Direct comparison
    direct_score = difflib.SequenceMatcher(None, target_norm, cross_norm).ratio()
    
    # Word-by-word comparison for complex names
    target_words = target_norm.split()
    cross_words = cross_norm.split()
    
    word_scores = []
    for t_word in target_words:
        if len(t_word) > 2:  # Skip very short words
            best_word_score = 0
            for c_word in cross_words:
                word_score = difflib.SequenceMatcher(None, t_word, c_word).ratio()
                best_word_score = max(best_word_score, word_score)
            word_scores.append(best_word_score)
    
    # Average word score
    avg_word_score = sum(word_scores) / len(word_scores) if word_scores else 0
    
    # Final score is max of direct and word-based matching
    final_score = max(direct_score, avg_word_score * 0.9)
    
    return final_score

# =============================================================================
# ENHANCED MATCHING LOGIC - REPLACES OLD int_contains FUNCTION
# =============================================================================

def enhanced_segment_match(req, geom, st_nm, fint, tint, segl, segid):
    """Enhanced matching with multiple strategies"""
    
    from_score = smart_intersection_match(req['from_st'], fint)
    to_score = smart_intersection_match(req['to_st'], tint)
    
    # Check both directions (from->to and to->from)
    forward_score = (from_score + to_score) / 2
    reverse_score = (smart_intersection_match(req['from_st'], tint) + 
                    smart_intersection_match(req['to_st'], fint)) / 2
    
    best_score = max(forward_score, reverse_score)
    direction = "Forward" if forward_score >= reverse_score else "Reverse"
    
    # Determine match quality
    if best_score >= 0.7:
        return {
            'match': True,
            'quality': 'High',
            'score': best_score,
            'method': f"Enhanced_{direction}",
            'details': f"From:{max(from_score, smart_intersection_match(req['from_st'], tint)):.2f}, To:{max(to_score, smart_intersection_match(req['to_st'], fint)):.2f}"
        }
    elif best_score >= 0.4:
        return {
            'match': True,
            'quality': 'Medium',
            'score': best_score,
            'method': f"Enhanced_Relaxed_{direction}",
            'details': f"From:{max(from_score, smart_intersection_match(req['from_st'], tint)):.2f}, To:{max(to_score, smart_intersection_match(req['to_st'], fint)):.2f}"
        }
    else:
        return {'match': False, 'score': best_score}

#– create output FC -----------------------------------------------------------#
ws          = os.path.dirname(split_path)
out_fc      = os.path.join(ws, f"{output_final_segments}.shp")
sr          = arcpy.Describe(split_path).spatialReference

if arcpy.Exists(out_fc): 
    arcpy.management.Delete(out_fc)

arcpy.management.CreateFeatureclass(ws, output_final_segments, "POLYLINE", spatial_reference=sr)

# Add fields with proper types - ADDED ENHANCED FIELDS
print(f"\n🔍 Finding matching segments for each table row:")
fields_to_add = [
    ("TableRow", "LONG"),
    ("Sec_ID", sec_type_out),
    ("StreetNm", "TEXT"),
    ("FromInt", "TEXT"),
    ("ToInt", "TEXT"),
    ("FromSt", "TEXT"),
    ("ToSt", "TEXT"),
    ("SegLen", "DOUBLE"),
    ("OrigSegID", "TEXT"),
    ("MatchStat", "TEXT"),
    ("MatchScor", "DOUBLE"),    # NEW: Match score
    ("MatchMthd", "TEXT"),      # NEW: Match method
    ("MatchDet", "TEXT")        # NEW: Match details
]

for field_name, field_type in fields_to_add:
    if field_type == "TEXT":
        arcpy.management.AddField(out_fc, field_name, field_type, field_length=100)
    else:
        arcpy.management.AddField(out_fc, field_name, field_type)

#– main matching loop - ENHANCED VERSION -------------------------------------#
row_match_counts, total_matches = {}, 0
unmatched = []

with arcpy.da.InsertCursor(out_fc,
        ["SHAPE@","TableRow","Sec_ID","StreetNm",
         "FromInt","ToInt","FromSt","ToSt","SegLen",
         "OrigSegID","MatchStat","MatchScor","MatchMthd","MatchDet"]) as icur:

    for req in table_reqs:
        row = req['row']
        
        if row <= 20:  # Show detailed processing for first 20 rows
            print(f"\n  Row {row}: Looking for segments connecting '{req['from_st']}' and '{req['to_st']}'")
        elif row % 100 == 0:  # Show progress every 100 rows
            progress = (row / len(table_reqs)) * 100
            print(f"\n  Progress: {progress:.1f}% complete (Row {row}/{len(table_reqs)})")

        matches = []
        with arcpy.da.SearchCursor(split_path,
                ["SHAPE@","RowNum","StreetName",
                 "FromInt","ToInt","SegLen","SegmentID"]) as scur:
            for geom,rn,st_nm,fint,tint,segl,segid in scur:
                if rn != row: 
                    continue
                
                # USE ENHANCED MATCHING INSTEAD OF OLD LOGIC
                match_result = enhanced_segment_match(req, geom, st_nm, fint, tint, segl, segid)
                
                if match_result['match']:
                    matches.append({
                        'geom': geom, 'st_nm': st_nm, 'fint': fint, 'tint': tint,
                        'segl': segl, 'segid': segid, 'result': match_result
                    })

        if matches:
            # Sort by score and take ONLY THE BEST MATCH
            matches.sort(key=lambda x: x['result']['score'], reverse=True)
            best_match = matches[0]  # Take only the highest scoring match
            
            if row <= 20:  # Show detailed results for first 20 rows
                print(f"    ✅ Found {len(matches)} candidates, selected best match")
                result = best_match['result']
                print(f"      SELECTED: {best_match['segid']}: {best_match['fint']} → {best_match['tint']} (Score: {result['score']:.3f}, {result['quality']})")
                if len(matches) > 1:
                    print(f"      (Skipped {len(matches)-1} lower-scoring alternatives)")
            
            # Insert ONLY the best match
            icur.insertRow([best_match['geom'],row,req['sec_id'],best_match['st_nm'],
                            best_match['fint'],best_match['tint'],req['from_st'],req['to_st'],
                            best_match['segl'],best_match['segid'],f"MATCHED_{best_match['result']['quality']}",
                            best_match['result']['score'],best_match['result']['method'],best_match['result']['details']])
            
            row_match_counts[row] = 1  # Always 1 match per row
            total_matches += 1
        else:
            if row <= 20:  # Show detailed results for first 20 rows
                print(f"    ❌ No matching segments found for Row {row}")
            unmatched.append(req)

#– export unmatched rows to a new table in the project geodatabase -----------#
if unmatched:
    # Create table in the project's default geodatabase
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    default_gdb = aprx.defaultGeodatabase
    un_tbl = os.path.join(default_gdb, unmatched_table_name)
    
    # Delete if exists
    if arcpy.Exists(un_tbl): 
        arcpy.management.Delete(un_tbl)
    
    # Create the table in the geodatabase
    arcpy.management.CreateTable(default_gdb, unmatched_table_name)
    
    # Add fields
    arcpy.management.AddField(un_tbl, "TableRow", "LONG")
    if sec_type_out == "TEXT":
        arcpy.management.AddField(un_tbl, "Sec_ID", sec_type_out, field_length=50)
    else:
        arcpy.management.AddField(un_tbl, "Sec_ID", sec_type_out)
    arcpy.management.AddField(un_tbl, "StreetNm", "TEXT", field_length=50)
    arcpy.management.AddField(un_tbl, "FromSt", "TEXT", field_length=50)
    arcpy.management.AddField(un_tbl, "ToSt", "TEXT", field_length=50)
    arcpy.management.AddField(un_tbl, "Reason", "TEXT", field_length=100)

    # Insert unmatched rows
    with arcpy.da.InsertCursor(
         un_tbl, ["TableRow","Sec_ID","StreetNm","FromSt","ToSt","Reason"]) as ucur:
        for r in unmatched:
            ucur.insertRow([r['row'], r['sec_id'], r['street'],
                            r['from_st'], r['to_st'],
                            "No enhanced match found"])

    # Add table to the current map
    m.addDataFromPath(un_tbl)
    print(f"\n⚠️  {len(unmatched)} unmatched rows exported to table: {unmatched_table_name} in project geodatabase")

#– add final layer & wrap up --------------------------------------------------#
m.addDataFromPath(out_fc)

print(f"\n✅ Created enhanced matched segments: {output_final_segments}")
print(f"Total matched segments: {total_matches}")

# ENHANCED SUMMARY WITH QUALITY BREAKDOWN
print(f"\n📊 Enhanced Summary by match quality:")
quality_counts = {}
with arcpy.da.SearchCursor(out_fc, ["MatchStat", "MatchScor"]) as cursor:
    for match_stat, score in cursor:
        quality_counts[match_stat] = quality_counts.get(match_stat, 0) + 1

for quality, count in sorted(quality_counts.items()):
    print(f"  - {quality}: {count} segments")

print(f"\n📈 Enhanced matching statistics:")
print(f"  - Rows with matches: {len(row_match_counts)}/{len(table_reqs)}")
print(f"  - Total matched segments: {total_matches}")
if len(row_match_counts) > 0:
    avg_segments = total_matches / len(row_match_counts)
    max_segments = max(row_match_counts.values())
    min_segments = min(row_match_counts.values())
    print(f"  - Average segments per matched row: {avg_segments:.1f}")
    print(f"  - Max segments (single row): {max_segments}")
    print(f"  - Min segments (single row): {min_segments}")

print(f"\n💡 New fields in output:")
print(f"  - MatchScor: Confidence score (0.0-1.0)")
print(f"  - MatchMthd: Enhanced matching method used")
print(f"  - MatchDet: Detailed scoring breakdown")
print(f"  - MatchStat: MATCHED_High/Medium quality indicator")

print("🎉 Enhanced matching complete!")

# =============================================================================
# COMPLETE WORKFLOW SUMMARY - MATCHING YOUR ORIGINAL CELL 6 ENDING
# =============================================================================

print("\\n" + "="*80)
print("🎉 COMPLETE ALL-ROWS WORKFLOW FINISHED SUCCESSFULLY!")
print("="*80)

print(f"\\n📋 LAYERS CREATED:")
print("1. ✅ CAMSCenterlines_Dissolved - All street centerlines dissolved by name")
print("2. ✅ AllRows_Matched_Segments - Street segments from entire table")
print("3. ✅ AllRows_Intersections - Intersection points for all matched streets")
print("4. ✅ AllRows_Split_Segments - All streets split into segments at intersections")
print("5. ✅ Final_Matched_Segments - Final segments matching table From/To requirements (ENHANCED)")

print(f"\\n📊 PROCESSING RESULTS:")
print(f"- Table rows processed: {len(table_reqs)}")
print(f"- Streets matched to centerlines: {len(matched_streets) if 'matched_streets' in locals() else 'N/A'}")
if 'intersection_count' in locals():
    print(f"- Total intersections found: {intersection_count}")
if 'total_created_segments' in locals():
    print(f"- Total split segments created: {total_created_segments}")
print(f"- Final matched segments: {total_matches}")

print(f"\\n💡 ENHANCED LAYER USAGE:")
print("- 'Final_Matched_Segments' contains your analysis-ready segments with enhanced matching")
print("- TableRow field identifies which table row each segment came from")
print("- FromSt/ToSt fields show the original table From/To values")
print("- FromInt/ToInt fields show the actual intersection names")
print("- MatchStat field indicates successful matches with quality levels")
print("- MatchScor field shows confidence score (0.0-1.0)")
print("- MatchMthd field shows which enhanced method was used")
print("- MatchDet field provides detailed scoring breakdown")

print(f"\\n📈 SUCCESS RATE:")
success_rate = (len(row_match_counts) / len(table_reqs)) * 100 if table_reqs else 0
print(f"- Successfully matched: {len(row_match_counts)}/{len(table_reqs)} rows ({success_rate:.1f}%)")

if len(unmatched) > 0:
    print(f"\\n⚠️  UNMATCHED ROWS: {len(unmatched)}")
    print(f"  - {len(unmatched)} rows could not be matched even with enhanced methods")
    print(f"  - Check street name spelling and intersection data")
    
    print(f"\\n💡 Possible reasons for unmatched rows:")
    print("  - Street name doesn't exist in centerlines")
    print("  - From/To streets don't intersect with the target street")
    print("  - Street name spelling differences")
    print("  - Missing intersection data")

print(f"\\n🔧 ENHANCED MATCHING IMPROVEMENTS:")
print("  - Smart intersection name parsing (handles 'Street A & Street B' format)")
print("  - Bidirectional matching (checks both from→to and to→from)")
print("  - Word-level similarity scoring for complex names")
print("  - Multiple confidence thresholds (High: 0.7+, Medium: 0.4+)")
print("  - Special handling for END segments")
print("  - Detailed scoring and method tracking")

print(f"\\n🎯 QUALITY CONTROL:")
print("  - Use MatchScor >= 0.7 for high-confidence segments")
print("  - Review MatchScor 0.4-0.7 for medium-confidence segments")
print("  - Check MatchDet field for detailed from/to scoring")
print("  - Filter by MatchStat for quality-based analysis")

print(f"\\n🗂️  All layers have been added to your ArcGIS Pro map!")
print("="*80)

Found Sec_ID field 'Sec_ID' with type: Integer
Mapped to output type: LONG
Processing 552 table requirements:

🔍 Finding matching segments for each table row:

  Row 1: Looking for segments connecting 'Albert St.' and 'Common Ave.'
    ✅ Found 4 candidates, selected best match
      SELECTED: Row1_Abbey St_Seg_2: Abbey St & Albert St → Abbey St & Common St (Score: 0.868, High)
      (Skipped 3 lower-scoring alternatives)

  Row 2: Looking for segments connecting 'Common Ave.' and 'Central Ave'
    ✅ Found 4 candidates, selected best match
      SELECTED: Row2_Abbey St_Seg_1: Abbey St & Common St → Abbey St & Central Ave (Score: 0.868, High)
      (Skipped 3 lower-scoring alternatives)

  Row 3: Looking for segments connecting 'Stimson Ave.' and 'Albert St.'
    ✅ Found 3 candidates, selected best match
      SELECTED: Row3_Abbey St_Seg_1: Abbey St & S Stimson Ave → Abbey St & Albert St (Score: 0.958, High)
      (Skipped 2 lower-scoring alternatives)

  Row 4: Looking for segments conn

  - Street name doesn't exist in centerlines
  - From/To streets don't intersect with the target street
  - Street name spelling differences
  - Missing intersection data
\n🔧 ENHANCED MATCHING IMPROVEMENTS:
  - Smart intersection name parsing (handles 'Street A & Street B' format)
  - Bidirectional matching (checks both from→to and to→from)
  - Word-level similarity scoring for complex names
  - Multiple confidence thresholds (High: 0.7+, Medium: 0.4+)
  - Special handling for END segments
  - Detailed scoring and method tracking
\n🎯 QUALITY CONTROL:
  - Use MatchScor >= 0.7 for high-confidence segments
  - Review MatchScor 0.4-0.7 for medium-confidence segments
  - Check MatchDet field for detailed from/to scoring
  - Filter by MatchStat for quality-based analysis
\n🗂️  All layers have been added to your ArcGIS Pro map!


In [34]:
# =============================================================================
# Alternative Matching Methods for Unmatched Rows
# =============================================================================
import arcpy
import os
import difflib

# Configuration
split_segments_layer = "AllRows_Split_Segments"
unmatched_table = "Unmatched_Rows"
output_alternative_segments = "Alternative_Matched_Segments"

project = arcpy.mp.ArcGISProject("CURRENT")
m = project.activeMap

# Get paths
split_path = next(arcpy.Describe(l).catalogPath for l in m.listLayers()
                  if l.name == split_segments_layer)
unmatched_path = next(arcpy.Describe(t).catalogPath for t in m.listTables()
                     if t.name == unmatched_table)

print("🔄 Alternative Matching Methods for Unmatched Rows")
print("=" * 60)

# Read unmatched rows
unmatched_rows = []
with arcpy.da.SearchCursor(unmatched_path, 
    ["TableRow", "Sec_ID", "StreetNm", "FromSt", "ToSt", "Reason"]) as cursor:
    for row in cursor:
        unmatched_rows.append({
            'row': row[0],
            'sec_id': row[1], 
            'street': row[2],
            'from_st': row[3],
            'to_st': row[4],
            'reason': row[5]
        })

print(f"Found {len(unmatched_rows)} unmatched rows to process")

# =============================================================================
# ALTERNATIVE MATCHING METHODS
# =============================================================================

def normalize_street_name(name):
    """Enhanced street name normalization"""
    if not name:
        return ""
    
    name = str(name).lower().strip()
    
    # Remove common prefixes/suffixes
    name = name.replace('n ', 'north ').replace('s ', 'south ')
    name = name.replace('e ', 'east ').replace('w ', 'west ')
    
    # Standardize abbreviations
    replacements = {
        ' avenue': ' ave', ' street': ' st', ' drive': ' dr',
        ' road': ' rd', ' boulevard': ' blvd', ' lane': ' ln',
        ' court': ' ct', ' place': ' pl', ' circle': ' cir',
        ' way': ' way', ' terrace': ' ter', ' parkway': ' pkwy'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    # Remove periods and extra spaces
    name = name.replace('.', '').replace(',', '')
    name = ' '.join(name.split())  # Remove extra whitespace
    
    return name

def fuzzy_match_intersections(target_intersection, available_intersections, threshold=0.6):
    """Fuzzy match intersection names"""
    target_norm = normalize_street_name(target_intersection)
    best_matches = []
    
    for intersection in available_intersections:
        intersection_norm = normalize_street_name(intersection)
        ratio = difflib.SequenceMatcher(None, target_norm, intersection_norm).ratio()
        
        if ratio >= threshold:
            best_matches.append((intersection, ratio))
    
    return sorted(best_matches, key=lambda x: x[1], reverse=True)

def method_1_exact_intersection_match(unmatched_req):
    """Method 1: Try exact intersection name matching with relaxed criteria"""
    matches = []
    
    with arcpy.da.SearchCursor(split_path,
            ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
        for geom, rn, st_nm, fint, tint, segl, segid in cursor:
            if rn != unmatched_req['row']:
                continue
                
            # Check if either intersection contains our target streets
            from_norm = normalize_street_name(unmatched_req['from_st'])
            to_norm = normalize_street_name(unmatched_req['to_st'])
            fint_norm = normalize_street_name(fint)
            tint_norm = normalize_street_name(tint)
            
            # More flexible matching - check if street names appear anywhere in intersection names
            from_in_fint = from_norm in fint_norm or any(word in fint_norm for word in from_norm.split())
            from_in_tint = from_norm in tint_norm or any(word in tint_norm for word in from_norm.split())
            to_in_fint = to_norm in fint_norm or any(word in fint_norm for word in to_norm.split())
            to_in_tint = to_norm in tint_norm or any(word in tint_norm for word in to_norm.split())
            
            if (from_in_fint or from_in_tint) and (to_in_fint or to_in_tint):
                matches.append((geom, st_nm, fint, tint, segl, segid, "Method1_Relaxed"))
    
    return matches

def method_2_fuzzy_intersection_match(unmatched_req):
    """Method 2: Fuzzy matching of intersection names"""
    matches = []
    
    # Get all available intersections for this row
    available_intersections = []
    with arcpy.da.SearchCursor(split_path,
            ["FromInt", "ToInt", "RowNum"]) as cursor:
        for fint, tint, rn in cursor:
            if rn == unmatched_req['row']:
                available_intersections.extend([fint, tint])
    
    available_intersections = list(set(available_intersections))  # Remove duplicates
    
    # Try fuzzy matching
    from_matches = fuzzy_match_intersections(unmatched_req['from_st'], available_intersections, 0.7)
    to_matches = fuzzy_match_intersections(unmatched_req['to_st'], available_intersections, 0.7)
    
    if from_matches and to_matches:
        # Find segments that connect the best fuzzy matches
        with arcpy.da.SearchCursor(split_path,
                ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
            for geom, rn, st_nm, fint, tint, segl, segid in cursor:
                if rn != unmatched_req['row']:
                    continue
                    
                for from_match, from_score in from_matches[:3]:  # Try top 3 matches
                    for to_match, to_score in to_matches[:3]:
                        if (from_match in [fint, tint]) and (to_match in [fint, tint]) and from_match != to_match:
                            avg_score = (from_score + to_score) / 2
                            matches.append((geom, st_nm, fint, tint, segl, segid, f"Method2_Fuzzy_{avg_score:.2f}"))
    
    return matches

def method_3_partial_match(unmatched_req):
    """Method 3: Find segments that contain at least one of the target intersections"""
    matches = []
    
    with arcpy.da.SearchCursor(split_path,
            ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
        for geom, rn, st_nm, fint, tint, segl, segid in cursor:
            if rn != unmatched_req['row']:
                continue
                
            from_norm = normalize_street_name(unmatched_req['from_st'])
            to_norm = normalize_street_name(unmatched_req['to_st'])
            fint_norm = normalize_street_name(fint)
            tint_norm = normalize_street_name(tint)
            
            # Check if at least one intersection matches
            from_match = from_norm in fint_norm or from_norm in tint_norm
            to_match = to_norm in fint_norm or to_norm in tint_norm
            
            if from_match or to_match:
                match_type = "Both" if (from_match and to_match) else ("From" if from_match else "To")
                matches.append((geom, st_nm, fint, tint, segl, segid, f"Method3_Partial_{match_type}"))
    
    return matches

def method_4_longest_segment(unmatched_req):
    """Method 4: Return the longest segment for this street as a fallback"""
    matches = []
    longest_segment = None
    max_length = 0
    
    with arcpy.da.SearchCursor(split_path,
            ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
        for geom, rn, st_nm, fint, tint, segl, segid in cursor:
            if rn != unmatched_req['row']:
                continue
                
            if segl > max_length:
                max_length = segl
                longest_segment = (geom, st_nm, fint, tint, segl, segid, "Method4_Longest")
    
    if longest_segment:
        matches.append(longest_segment)
    
    return matches

# =============================================================================
# CREATE OUTPUT FEATURE CLASS
# =============================================================================

workspace = os.path.dirname(split_path)
output_path = os.path.join(workspace, f"{output_alternative_segments}.shp")
sr = arcpy.Describe(split_path).spatialReference

if arcpy.Exists(output_path):
    arcpy.management.Delete(output_path)

arcpy.management.CreateFeatureclass(workspace, output_alternative_segments, "POLYLINE", spatial_reference=sr)

# Add fields
fields_to_add = [
    ("TableRow", "LONG"),
    ("Sec_ID", "TEXT"),
    ("StreetNm", "TEXT"),
    ("FromInt", "TEXT"),
    ("ToInt", "TEXT"),
    ("FromSt", "TEXT"),
    ("ToSt", "TEXT"),
    ("SegLen", "DOUBLE"),
    ("OrigSegID", "TEXT"),
    ("MatchMthd", "TEXT"),
    ("Confidence", "TEXT")
]

for field_name, field_type in fields_to_add:
    if field_type == "TEXT":
        arcpy.management.AddField(output_path, field_name, field_type, field_length=100)
    else:
        arcpy.management.AddField(output_path, field_name, field_type)

# =============================================================================
# PROCESS UNMATCHED ROWS WITH ALTERNATIVE METHODS
# =============================================================================

total_alternative_matches = 0
method_success_counts = {"Method1": 0, "Method2": 0, "Method3": 0, "Method4": 0}

print(f"\n🔍 Trying alternative matching methods:")

with arcpy.da.InsertCursor(output_path,
        ["SHAPE@", "TableRow", "Sec_ID", "StreetNm", "FromInt", "ToInt", 
         "FromSt", "ToSt", "SegLen", "OrigSegID", "MatchMthd", "Confidence"]) as insert_cursor:
    
    for i, req in enumerate(unmatched_rows):
        
        if i < 10:  # Show details for first 10
            print(f"\n  Row {req['row']}: {req['street']} from {req['from_st']} to {req['to_st']}")
        elif i % 50 == 0:  # Progress every 50 rows
            progress = (i / len(unmatched_rows)) * 100
            print(f"\n  Progress: {progress:.1f}% ({i}/{len(unmatched_rows)})")
        
        # Try each method in order
        matches_found = False
        
        # Method 1: Relaxed exact matching
        matches = method_1_exact_intersection_match(req)
        if matches:
            for geom, st_nm, fint, tint, segl, segid, method in matches:
                insert_cursor.insertRow([
                    geom, req['row'], str(req['sec_id']), st_nm, fint, tint,
                    req['from_st'], req['to_st'], segl, segid, method, "Medium"
                ])
                total_alternative_matches += 1
            method_success_counts["Method1"] += 1
            matches_found = True
            if i < 10:
                print(f"    ✅ Method 1 (Relaxed): Found {len(matches)} segment(s)")
        
        # Method 2: Fuzzy matching (only if Method 1 failed)
        if not matches_found:
            matches = method_2_fuzzy_intersection_match(req)
            if matches:
                for geom, st_nm, fint, tint, segl, segid, method in matches:
                    insert_cursor.insertRow([
                        geom, req['row'], str(req['sec_id']), st_nm, fint, tint,
                        req['from_st'], req['to_st'], segl, segid, method, "Low"
                    ])
                    total_alternative_matches += 1
                method_success_counts["Method2"] += 1
                matches_found = True
                if i < 10:
                    print(f"    ✅ Method 2 (Fuzzy): Found {len(matches)} segment(s)")
        
        # Method 3: Partial matching (only if previous methods failed)
        if not matches_found:
            matches = method_3_partial_match(req)
            if matches:
                for geom, st_nm, fint, tint, segl, segid, method in matches:
                    insert_cursor.insertRow([
                        geom, req['row'], str(req['sec_id']), st_nm, fint, tint,
                        req['from_st'], req['to_st'], segl, segid, method, "Low"
                    ])
                    total_alternative_matches += 1
                method_success_counts["Method3"] += 1
                matches_found = True
                if i < 10:
                    print(f"    ✅ Method 3 (Partial): Found {len(matches)} segment(s)")
        
        # Method 4: Longest segment fallback (only if all others failed)
        if not matches_found:
            matches = method_4_longest_segment(req)
            if matches:
                for geom, st_nm, fint, tint, segl, segid, method in matches:
                    insert_cursor.insertRow([
                        geom, req['row'], str(req['sec_id']), st_nm, fint, tint,
                        req['from_st'], req['to_st'], segl, segid, method, "Very Low"
                    ])
                    total_alternative_matches += 1
                method_success_counts["Method4"] += 1
                matches_found = True
                if i < 10:
                    print(f"    ⚠️ Method 4 (Fallback): Found {len(matches)} segment(s)")
        
        if not matches_found and i < 10:
            print(f"    ❌ No matches found with any method")

# Add to map
m.addDataFromPath(output_path)

# =============================================================================
# SUMMARY REPORT
# =============================================================================

print(f"\n" + "="*80)
print(f"🎯 ALTERNATIVE MATCHING RESULTS")
print(f"="*80)

print(f"\n📊 Overall Results:")
print(f"  • Total unmatched rows processed: {len(unmatched_rows)}")
print(f"  • Rows matched with alternative methods: {sum(method_success_counts.values())}")
print(f"  • Total alternative segments found: {total_alternative_matches}")
success_rate = (sum(method_success_counts.values()) / len(unmatched_rows)) * 100 if unmatched_rows else 0
print(f"  • Alternative matching success rate: {success_rate:.1f}%")

print(f"\n🔧 Method Performance:")
for method, count in method_success_counts.items():
    percentage = (count / len(unmatched_rows)) * 100 if unmatched_rows else 0
    print(f"  • {method}: {count} rows ({percentage:.1f}%)")

print(f"\n💡 Method Descriptions:")
print(f"  • Method 1 (Relaxed): More flexible intersection name matching")
print(f"  • Method 2 (Fuzzy): Uses similarity scoring for intersection names") 
print(f"  • Method 3 (Partial): Matches segments with at least one correct intersection")
print(f"  • Method 4 (Fallback): Returns longest segment when all else fails")

print(f"\n🎯 Confidence Levels:")
print(f"  • Medium: Good alternative match, likely correct")
print(f"  • Low: Possible match, requires manual review")
print(f"  • Very Low: Fallback match, definitely needs manual review")

remaining_unmatched = len(unmatched_rows) - sum(method_success_counts.values())
if remaining_unmatched > 0:
    print(f"\n⚠️ Still Unmatched: {remaining_unmatched} rows")
    print(f"  These may require manual intervention or data correction")

print(f"\n🗂️ Output Layer: '{output_alternative_segments}' added to map")
print(f"   Use the 'MatchMthd' and 'Confidence' fields to review results")
print(f"="*80)

🔄 Alternative Matching Methods for Unmatched Rows
Found 55 unmatched rows to process

🔍 Trying alternative matching methods:

  Row 22: AL N/Amar from Gilwood Ave. to Fickewirth Ave.
    ❌ No matches found with any method

  Row 40: Amar Frontage S from End to Sunkist Ave.
    ❌ No matches found with any method

  Row 41: Amar Frontage S from Tamar Dr. to Sunkist Ave.
    ❌ No matches found with any method

  Row 73: Ballista Ave. from Blackwood St. to Cadwell St.
    ✅ Method 1 (Relaxed): Found 2 segment(s)

  Row 74: Ballista Ave. from Lawnwood St. to Amar Rd.
    ⚠️ Method 4 (Fallback): Found 1 segment(s)

  Row 75: Ballista Ave. from Mulvane St. to Loukelton St.
    ✅ Method 1 (Relaxed): Found 2 segment(s)

  Row 97: Beckner St. Cul-De-Sac from END to Beckner St.
    ❌ No matches found with any method

  Row 133: Common Ave. from Central Ave. to Old Valley Blvd.
    ⚠️ Method 4 (Fallback): Found 1 segment(s)

  Row 172: Fifth St. from Rowland St. to Workman St.
    ❌ No matches fou

In [35]:
# =============================================================================
# Alternative Matching Methods for Unmatched Rows
# =============================================================================
import arcpy
import os
import difflib

# Configuration
split_segments_layer = "AllRows_Split_Segments"
unmatched_table = "Unmatched_Rows"
output_alternative_segments = "Alternative_Matched_Segments"

project = arcpy.mp.ArcGISProject("CURRENT")
m = project.activeMap

# Get paths
split_path = next(arcpy.Describe(l).catalogPath for l in m.listLayers()
                  if l.name == split_segments_layer)
unmatched_path = next(arcpy.Describe(t).catalogPath for t in m.listTables()
                     if t.name == unmatched_table)

print("🔄 Alternative Matching Methods for Unmatched Rows")
print("=" * 60)

# Read unmatched rows
unmatched_rows = []
with arcpy.da.SearchCursor(unmatched_path, 
    ["TableRow", "Sec_ID", "StreetNm", "FromSt", "ToSt", "Reason"]) as cursor:
    for row in cursor:
        unmatched_rows.append({
            'row': row[0],
            'sec_id': row[1], 
            'street': row[2],
            'from_st': row[3],
            'to_st': row[4],
            'reason': row[5]
        })

print(f"Found {len(unmatched_rows)} unmatched rows to process")

# =============================================================================
# ALTERNATIVE MATCHING METHODS
# =============================================================================

def normalize_street_name(name):
    """Enhanced street name normalization"""
    if not name:
        return ""
    
    name = str(name).lower().strip()
    
    # Remove common prefixes/suffixes
    name = name.replace('n ', 'north ').replace('s ', 'south ')
    name = name.replace('e ', 'east ').replace('w ', 'west ')
    
    # Standardize abbreviations
    replacements = {
        ' avenue': ' ave', ' street': ' st', ' drive': ' dr',
        ' road': ' rd', ' boulevard': ' blvd', ' lane': ' ln',
        ' court': ' ct', ' place': ' pl', ' circle': ' cir',
        ' way': ' way', ' terrace': ' ter', ' parkway': ' pkwy'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    # Remove periods and extra spaces
    name = name.replace('.', '').replace(',', '')
    name = ' '.join(name.split())  # Remove extra whitespace
    
    return name

def fuzzy_match_intersections(target_intersection, available_intersections, threshold=0.6):
    """Fuzzy match intersection names"""
    target_norm = normalize_street_name(target_intersection)
    best_matches = []
    
    for intersection in available_intersections:
        intersection_norm = normalize_street_name(intersection)
        ratio = difflib.SequenceMatcher(None, target_norm, intersection_norm).ratio()
        
        if ratio >= threshold:
            best_matches.append((intersection, ratio))
    
    return sorted(best_matches, key=lambda x: x[1], reverse=True)

def method_1_exact_intersection_match(unmatched_req):
    """Method 1: Try exact intersection name matching with relaxed criteria"""
    matches = []
    
    with arcpy.da.SearchCursor(split_path,
            ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
        for geom, rn, st_nm, fint, tint, segl, segid in cursor:
            if rn != unmatched_req['row']:
                continue
                
            # Check if either intersection contains our target streets
            from_norm = normalize_street_name(unmatched_req['from_st'])
            to_norm = normalize_street_name(unmatched_req['to_st'])
            fint_norm = normalize_street_name(fint)
            tint_norm = normalize_street_name(tint)
            
            # More flexible matching - check if street names appear anywhere in intersection names
            from_in_fint = from_norm in fint_norm or any(word in fint_norm for word in from_norm.split())
            from_in_tint = from_norm in tint_norm or any(word in tint_norm for word in from_norm.split())
            to_in_fint = to_norm in fint_norm or any(word in fint_norm for word in to_norm.split())
            to_in_tint = to_norm in tint_norm or any(word in tint_norm for word in to_norm.split())
            
            if (from_in_fint or from_in_tint) and (to_in_fint or to_in_tint):
                matches.append((geom, st_nm, fint, tint, segl, segid, "Method1_Relaxed"))
    
    return matches

def method_2_fuzzy_intersection_match(unmatched_req):
    """Method 2: Fuzzy matching of intersection names"""
    matches = []
    
    # Get all available intersections for this row
    available_intersections = []
    with arcpy.da.SearchCursor(split_path,
            ["FromInt", "ToInt", "RowNum"]) as cursor:
        for fint, tint, rn in cursor:
            if rn == unmatched_req['row']:
                available_intersections.extend([fint, tint])
    
    available_intersections = list(set(available_intersections))  # Remove duplicates
    
    # Try fuzzy matching
    from_matches = fuzzy_match_intersections(unmatched_req['from_st'], available_intersections, 0.7)
    to_matches = fuzzy_match_intersections(unmatched_req['to_st'], available_intersections, 0.7)
    
    if from_matches and to_matches:
        # Find segments that connect the best fuzzy matches
        with arcpy.da.SearchCursor(split_path,
                ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
            for geom, rn, st_nm, fint, tint, segl, segid in cursor:
                if rn != unmatched_req['row']:
                    continue
                    
                for from_match, from_score in from_matches[:3]:  # Try top 3 matches
                    for to_match, to_score in to_matches[:3]:
                        if (from_match in [fint, tint]) and (to_match in [fint, tint]) and from_match != to_match:
                            avg_score = (from_score + to_score) / 2
                            matches.append((geom, st_nm, fint, tint, segl, segid, f"Method2_Fuzzy_{avg_score:.2f}"))
    
    return matches

def method_3_partial_match(unmatched_req):
    """Method 3: Find segments that contain at least one of the target intersections"""
    matches = []
    
    with arcpy.da.SearchCursor(split_path,
            ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
        for geom, rn, st_nm, fint, tint, segl, segid in cursor:
            if rn != unmatched_req['row']:
                continue
                
            from_norm = normalize_street_name(unmatched_req['from_st'])
            to_norm = normalize_street_name(unmatched_req['to_st'])
            fint_norm = normalize_street_name(fint)
            tint_norm = normalize_street_name(tint)
            
            # Check if at least one intersection matches
            from_match = from_norm in fint_norm or from_norm in tint_norm
            to_match = to_norm in fint_norm or to_norm in tint_norm
            
            if from_match or to_match:
                match_type = "Both" if (from_match and to_match) else ("From" if from_match else "To")
                matches.append((geom, st_nm, fint, tint, segl, segid, f"Method3_Partial_{match_type}"))
    
    return matches

def method_4_longest_segment(unmatched_req):
    """Method 4: Return the longest segment for this street as a fallback"""
    matches = []
    longest_segment = None
    max_length = 0
    
    with arcpy.da.SearchCursor(split_path,
            ["SHAPE@", "RowNum", "StreetName", "FromInt", "ToInt", "SegLen", "SegmentID"]) as cursor:
        for geom, rn, st_nm, fint, tint, segl, segid in cursor:
            if rn != unmatched_req['row']:
                continue
                
            if segl > max_length:
                max_length = segl
                longest_segment = (geom, st_nm, fint, tint, segl, segid, "Method4_Longest")
    
    if longest_segment:
        matches.append(longest_segment)
    
    return matches

# =============================================================================
# CREATE OUTPUT FEATURE CLASS
# =============================================================================

workspace = os.path.dirname(split_path)
output_path = os.path.join(workspace, f"{output_alternative_segments}.shp")
sr = arcpy.Describe(split_path).spatialReference

if arcpy.Exists(output_path):
    arcpy.management.Delete(output_path)

arcpy.management.CreateFeatureclass(workspace, output_alternative_segments, "POLYLINE", spatial_reference=sr)

# Add fields
fields_to_add = [
    ("TableRow", "LONG"),
    ("Sec_ID", "TEXT"),
    ("StreetNm", "TEXT"),
    ("FromInt", "TEXT"),
    ("ToInt", "TEXT"),
    ("FromSt", "TEXT"),
    ("ToSt", "TEXT"),
    ("SegLen", "DOUBLE"),
    ("OrigSegID", "TEXT"),
    ("MatchMthd", "TEXT"),
    ("Confidence", "TEXT")
]

for field_name, field_type in fields_to_add:
    if field_type == "TEXT":
        arcpy.management.AddField(output_path, field_name, field_type, field_length=100)
    else:
        arcpy.management.AddField(output_path, field_name, field_type)

# =============================================================================
# PROCESS UNMATCHED ROWS - SELECT BEST MATCH FOR EACH ROW
# =============================================================================

total_alternative_matches = 0
method_success_counts = {"Method1": 0, "Method2": 0, "Method3": 0, "Method4": 0}

print(f"\n🔍 Finding best alternative match for each unmatched row:")

with arcpy.da.InsertCursor(output_path,
        ["SHAPE@", "TableRow", "Sec_ID", "StreetNm", "FromInt", "ToInt", 
         "FromSt", "ToSt", "SegLen", "OrigSegID", "MatchMthd", "Confidence"]) as insert_cursor:
    
    for i, req in enumerate(unmatched_rows):
        
        if i < 10:  # Show details for first 10
            print(f"\n  Row {req['row']}: {req['street']} from {req['from_st']} to {req['to_st']}")
        elif i % 50 == 0:  # Progress every 50 rows
            progress = (i / len(unmatched_rows)) * 100
            print(f"\n  Progress: {progress:.1f}% ({i}/{len(unmatched_rows)})")
        
        best_match = None
        best_method = None
        best_confidence = None
        
        # Try Method 1: Relaxed exact matching - PICK BEST MATCH
        matches = method_1_exact_intersection_match(req)
        if matches:
            # If multiple matches, pick the one with most reasonable length
            best_match = max(matches, key=lambda x: x[4])  # Pick longest segment
            best_method = "Method1_Relaxed"
            best_confidence = "Medium"
            method_success_counts["Method1"] += 1
            if i < 10:
                print(f"    ✅ Method 1 (Relaxed): Selected best of {len(matches)} matches")
        
        # Method 2: Fuzzy matching (only if Method 1 failed) - PICK BEST MATCH
        elif not best_match:
            matches = method_2_fuzzy_intersection_match(req)
            if matches:
                # Pick match with highest confidence score (embedded in method name)
                best_match = matches[0]  # Already sorted by score in the method
                best_method = matches[0][6]  # Method name with score
                best_confidence = "Low"
                method_success_counts["Method2"] += 1
                if i < 10:
                    print(f"    ✅ Method 2 (Fuzzy): Selected best of {len(matches)} matches")
        
        # Method 3: Partial matching (only if previous methods failed) - PICK BEST MATCH
        elif not best_match:
            matches = method_3_partial_match(req)
            if matches:
                # Prefer "Both" matches over "From" or "To", then pick longest
                both_matches = [m for m in matches if "Both" in m[6]]
                if both_matches:
                    best_match = max(both_matches, key=lambda x: x[4])
                else:
                    best_match = max(matches, key=lambda x: x[4])
                best_method = best_match[6]
                best_confidence = "Low"
                method_success_counts["Method3"] += 1
                if i < 10:
                    print(f"    ✅ Method 3 (Partial): Selected best of {len(matches)} matches")
        
        # Method 4: Longest segment fallback (only if all others failed)
        elif not best_match:
            matches = method_4_longest_segment(req)
            if matches:
                best_match = matches[0]  # Only returns one match anyway
                best_method = "Method4_Longest"
                best_confidence = "Very Low"
                method_success_counts["Method4"] += 1
                if i < 10:
                    print(f"    ⚠️ Method 4 (Fallback): Using longest segment")
        
        # Insert the single best match for this row
        if best_match:
            geom, st_nm, fint, tint, segl, segid = best_match[:6]
            insert_cursor.insertRow([
                geom, req['row'], str(req['sec_id']), st_nm, fint, tint,
                req['from_st'], req['to_st'], segl, segid, best_method, best_confidence
            ])
            total_alternative_matches += 1
            
            if i < 10:
                print(f"    📍 Selected: {fint} → {tint} ({segl:.1f} units)")
        else:
            if i < 10:
                print(f"    ❌ No matches found with any method")

# Add to map
m.addDataFromPath(output_path)

# =============================================================================
# SUMMARY REPORT
# =============================================================================

print(f"\n" + "="*80)
print(f"🎯 ALTERNATIVE MATCHING RESULTS")
print(f"="*80)

print(f"\n📊 Overall Results:")
print(f"  • Total unmatched rows processed: {len(unmatched_rows)}")
print(f"  • Rows matched with alternative methods: {total_alternative_matches}")
print(f"  • Expected matches: {len(unmatched_rows)} (1 per unmatched row)")
success_rate = (total_alternative_matches / len(unmatched_rows)) * 100 if unmatched_rows else 0
print(f"  • Alternative matching success rate: {success_rate:.1f}%")

if total_alternative_matches == len(unmatched_rows):
    print(f"  ✅ Perfect! Every unmatched row now has exactly one best match")
elif total_alternative_matches < len(unmatched_rows):
    still_unmatched = len(unmatched_rows) - total_alternative_matches
    print(f"  ⚠️ {still_unmatched} rows still couldn't be matched with any method")

print(f"\n🔧 Method Performance:")
for method, count in method_success_counts.items():
    percentage = (count / len(unmatched_rows)) * 100 if unmatched_rows else 0
    print(f"  • {method}: {count} rows ({percentage:.1f}%)")

print(f"\n💡 Method Descriptions:")
print(f"  • Method 1 (Relaxed): More flexible intersection name matching")
print(f"  • Method 2 (Fuzzy): Uses similarity scoring for intersection names") 
print(f"  • Method 3 (Partial): Matches segments with at least one correct intersection")
print(f"  • Method 4 (Fallback): Returns longest segment when all else fails")

print(f"\n🎯 Confidence Levels:")
print(f"  • Medium: Good alternative match, likely correct")
print(f"  • Low: Possible match, requires manual review")
print(f"  • Very Low: Fallback match, definitely needs manual review")

print(f"\n🗂️ Output Layer: '{output_alternative_segments}' added to map")
print(f"   • Contains exactly {total_alternative_matches} segments")
print(f"   • Use 'MatchMthd' and 'Confidence' fields to review results")
print(f"   • Each unmatched row has at most one alternative match")
print(f"="*80)

🔄 Alternative Matching Methods for Unmatched Rows
Found 55 unmatched rows to process

🔍 Finding best alternative match for each unmatched row:

  Row 22: AL N/Amar from Gilwood Ave. to Fickewirth Ave.
    ❌ No matches found with any method

  Row 40: Amar Frontage S from End to Sunkist Ave.
    ❌ No matches found with any method

  Row 41: Amar Frontage S from Tamar Dr. to Sunkist Ave.
    ❌ No matches found with any method

  Row 73: Ballista Ave. from Blackwood St. to Cadwell St.
    ✅ Method 1 (Relaxed): Selected best of 2 matches
    📍 Selected: Ballista Ave & Fairgrove Ave → Ballista Ave & (162.0 units)

  Row 74: Ballista Ave. from Lawnwood St. to Amar Rd.
    ❌ No matches found with any method

  Row 75: Ballista Ave. from Mulvane St. to Loukelton St.
    ✅ Method 1 (Relaxed): Selected best of 2 matches
    📍 Selected: Ballista Ave & Fairgrove Ave → Ballista Ave & (162.0 units)

  Row 97: Beckner St. Cul-De-Sac from END to Beckner St.
    ❌ No matches found with any method

  Ro